In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Carregar dataset
df = pd.read_csv('games.csv')

# Filtrar temporadas a partir de 2013 e jogos da temporada regular
df = df[df['season'] >= 2013]
df = df[df['game_type'] == 'REG']

# Definir o vencedor (1 se o time da casa vencer, 0 se o time visitante vencer)
df['winner'] = df['result'].apply(lambda x: 1 if x > 0 else 0)
df = df[df['result'] != 0]

# Contar vitórias do time da casa
df['home_team_wins'] = df.groupby(['home_team', 'season'])['winner'].cumsum()

# Contar vitórias do time visitante (inverso das vitórias do time da casa)
df['away_team_wins'] = df.groupby(['away_team', 'season'])['winner'].transform(lambda x: (1 - x).cumsum())

# Contar derrotas do time da casa (inverso das vitórias do time da casa)
df['home_team_losses'] = df.groupby(['home_team', 'season'])['winner'].transform(lambda x: (1 - x).cumsum())

# Contar derrotas do time visitante
df['away_team_losses'] = df.groupby(['away_team', 'season'])['winner'].cumsum()

# Selecionar as colunas de interesse para o modelo
parametros = ['home_team_wins', 'away_team_wins', 'home_team_losses', 'away_team_losses']

# Dividir em treino e teste
df_train = df[df['season'] < 2023]
df_test = df[df['season'] >= 2023]

X_train = df_train[parametros]
y_train = df_train['winner']
X_test = df_test[parametros]
y_test = df_test['winner']

# Criar o modelo de regressão logística
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

# Fazer previsões
y_pred = modelo.predict(X_test)

# Avaliar a acurácia do modelo
acuracia = accuracy_score(y_test, y_pred)
print(f'Acurácia: {acuracia * 100:.2f}%')


Acurácia: 85.48%


In [2]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(modelo, X_train, y_train, cv=5)  # 5-fold cross-validation
print(f'Acurácia média com cross-validation: {scores.mean() * 100:.2f}%')


Acurácia média com cross-validation: 84.38%


In [ ]:
#Fazer previsões
y_pred = modelo.predict(X_test)

# Criar um DataFrame com as previsões e os times envolvidos
df_predictions = df_test[['away_team', 'home_team']].copy()
df_predictions['predicted_winner'] = y_pred

# Função para exibir a predição com os nomes dos times
def display_prediction(row):
    if row['predicted_winner'] == 1:
        return f"{row['home_team']} vs {row['away_team']}, vencedor: {row['home_team']}"
    else:
        return f"{row['home_team']} vs {row['away_team']}, vencedor: {row['away_team']}"

# Aplicar a função no DataFrame de predições
df_predictions['prediction_display'] = df_predictions.apply(display_prediction, axis=1)

# Exibir as predições
for pred in df_predictions['prediction_display']:
    print(pred)


In [ ]:
# Filtrar partidas envolvendo o time "BAL"
df_bal_games = df_test[(df_test['away_team'] == 'BAL') | (df_test['home_team'] == 'BAL')].copy()

# Fazer previsões
y_pred_bal = modelo.predict(df_bal_games[parametros])

# Criar um DataFrame com as previsões e os times envolvidos
df_bal_games['predicted_winner'] = y_pred_bal

# Função para exibir a predição com os nomes dos times
def display_prediction(row):
    if row['predicted_winner'] == 1:
        return f"{row['home_team']} vs {row['away_team']}, vencedor: {row['home_team']}"
    else:
        return f"{row['home_team']} vs {row['away_team']}, vencedor: {row['away_team']}"

# Aplicar a função no DataFrame de predições
df_bal_games['prediction_display'] = df_bal_games.apply(display_prediction, axis=1)

# Exibir as predições apenas para jogos do "BAL"
for pred in df_bal_games['prediction_display']:
    print(pred)


In [ ]:
# Filtrar partidas envolvendo o time "BAL" na temporada de 2024
df_bal_2024_games = df_test[((df_test['away_team'] == 'BAL') | (df_test['home_team'] == 'BAL')) & (df_test['season'] == 2024)].copy()

# Fazer previsões
y_pred_bal_2024 = modelo.predict(df_bal_2024_games[parametros])

# Criar um DataFrame com as previsões e os times envolvidos
df_bal_2024_games['predicted_winner'] = y_pred_bal_2024

# Função para exibir a predição com os nomes dos times
def display_prediction(row):
    if row['predicted_winner'] == 1:
        return f"{row['home_team']} vs {row['away_team']}, vencedor: {row['home_team']}"
    else:
        return f"{row['home_team']} vs {row['away_team']}, vencedor: {row['away_team']}"

# Aplicar a função no DataFrame de predições
df_bal_2024_games['prediction_display'] = df_bal_2024_games.apply(display_prediction, axis=1)

# Exibir as predições apenas para jogos do "BAL" na temporada de 2024
for pred in df_bal_2024_games['prediction_display']:
    print(pred)


In [ ]:
# Variável para o número da semana que você deseja visualizar
semana = int(input("Digite o número da semana (por exemplo, 1 para semana 1): "))

# Filtrar os jogos da temporada de 2024 para a semana desejada
df_semana = df_test[(df_test['season'] == 2024) & (df_test['week'] == semana)].copy()

# Fazer previsões
y_pred_semana = modelo.predict(df_semana[parametros])

# Adicionar a coluna de vencedor previsto
df_semana['predicted_winner'] = y_pred_semana

# Função para exibir a predição com os nomes dos times
def display_prediction(row):
    if row['predicted_winner'] == 1:
        return f"{row['home_team']} vs {row['away_team']}, vencedor: {row['home_team']}"
    else:
        return f"{row['home_team']} vs {row['away_team']}, vencedor: {row['away_team']}"

# Aplicar a função no DataFrame de predições
df_semana['prediction_display'] = df_semana.apply(display_prediction, axis=1)

# Exibir as predições para todos os jogos da semana escolhida
for pred in df_semana['prediction_display']:
    print(pred)


In [7]:
# Função para calcular lucro com base nas odds e predições
def calcular_lucro(row):
    aposta = 100  # Valor fixo da aposta em cada jogo
    if row['predicted_winner'] == 1:  # Aposta no time da casa
        odds = row['home_moneyline']
        if row['winner'] == 1:  # Aposta correta
            return aposta * (odds / 100) if odds > 0 else aposta * (100 / abs(odds))
        else:  # Aposta errada
            return -aposta
    else:  # Aposta no time visitante
        odds = row['away_moneyline']
        if row['winner'] == 0:  # Aposta correta
            return aposta * (odds / 100) if odds > 0 else aposta * (100 / abs(odds))
        else:  # Aposta errada
            return -aposta

# Adicionar colunas de odds no DataFrame de teste
df_test['predicted_winner'] = y_pred
df_test['lucro'] = df_test.apply(calcular_lucro, axis=1)

# Calcular lucro total e lucro médio por jogo
lucro_total = df_test['lucro'].sum()
lucro_medio = df_test['lucro'].mean()

print(f"Lucro total: ${lucro_total:.2f}")
print(f"Lucro médio por jogo: ${lucro_medio:.2f}")


Lucro total: $23936.46
Lucro médio por jogo: $50.61


C:\Users\felipe\AppData\Local\Temp\ipykernel_12672\1411890425.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predicted_winner'] = y_pred
C:\Users\felipe\AppData\Local\Temp\ipykernel_12672\1411890425.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['lucro'] = df_test.apply(calcular_lucro, axis=1)
